## Work
1. 試比較有 BN 在 Batch_size = 2, 16, 32, 128, 256 下的差異
2. 請嘗試將 BN 放在 Activation 之前，並比較訓練結果
3. 請於 BN 放在 Input Layer 後，並比較結果

In [1]:
import os
import keras
import itertools
# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
from keras.layers import BatchNormalization
def build_mlp(input_shape, output_units=10, n_neurons=[512,256,128]):
    input_layer = keras.layers.Input(input_shape)
    for i, n_units in enumerate(n_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, activation='relu', name='hidden_layer'+str(i))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_units, activation='relu', name='hidden_layer'+str(i))(x)
            x = BatchNormalization()(x)
    
    out = keras.layers.Dense(units=output_units, activation='softmax', name='output')(x)
    model = keras.models.Model(inputs=input_layer, outputs=out)
    
    return model

In [6]:
LEARNING_RATE = 1e-3
EPOCHS = 10
BATCH_SIZE = [2, 16, 32, 128, 256]
MOMENTUM = 0.95

In [ ]:
results = {}
for i, bat_size in enumerate(BATCH_SIZE):
    print('Experiment with batch size-%s' % str(bat_size))
    model = build_mlp(input_shape=x_train.shape[1:])
    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.fit(x_train, y_train, batch_size=bat_size, epochs=EPOCHS, validation_data=(x_test, y_test), shuffle=True)

    train_loss = model.history.history['loss']
    valid_loss = model.history.history['val_loss']
    train_acc = model.history.history['accuracy']
    valid_acc = model.history.history['val_accuracy']

    exp_name_tag = 'batch size=-%s' % str(bat_size)
    results[exp_name_tag] = {'train-loss': train_loss,
                             'val-loss': valid_loss,
                             'train-acc': train_acc,
                             'val-acc': valid_acc}

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.colors as colors
import matplotlib.cm as mplcm

num_colors = 5
c_Norm = colors.Normalize(vmin=0, vmax=num_colors-1)
Scalor_map = mplcm.ScalarMappable(norm=c_Norm, cmap=plt.get_cmap('gist_rainbow'))
color_bar = [Scalor_map.to_rgba(i) for i in range(num_colors)]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])), results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['val-loss'])), results[cond]['val-loss'], '--', label=cond, color=color_bar[i])
plt.title('LOSS')
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])), results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['val-acc'])), results[cond]['val-acc'], '--', label=cond, color=color_bar[i])
plt.title('ACCURACY')
plt.legend()
plt.show()


In [ ]:
from keras.layers import BatchNormalization
def build_mlp1(input_shape, output_units=10, n_neurons=[512,256,128]):
    input_layer = keras.layers.Input(input_shape)
    for i, n_units in enumerate(n_neurons):
        if i == 0:
            x = BatchNormalization()(input_layer)
            x = keras.layers.Dense(units=n_units, activation='relu', name='hidden_layer'+str(i))(x)
        else:
            x = BatchNormalization()(x)
            x = keras.layers.Dense(units=n_units, activation='relu', name='hidden_layer'+str(i))(x)
    
    out = keras.layers.Dense(units=output_units, activation='softmax', name='output')(x)
    model = keras.models.Model(inputs=input_layer, outputs=out)
    
    return model

In [ ]:
results1 = {}
for i, bat_size in enumerate(BATCH_SIZE):
    print('Experiment with batch size-%s' % str(bat_size))
    model = build_mlp1(input_shape=x_train.shape[1:])
    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.fit(x_train, y_train, batch_size=bat_size, epochs=EPOCHS, validation_data=(x_test, y_test), shuffle=True)

    train_loss = model.history.history['loss']
    valid_loss = model.history.history['val_loss']
    train_acc = model.history.history['accuracy']
    valid_acc = model.history.history['val_accuracy']

    exp_name_tag = 'batch size=-%s' % str(bat_size)
    results1[exp_name_tag] = {'train-loss': train_loss,
                             'val-loss': valid_loss,
                             'train-acc': train_acc,
                             'val-acc': valid_acc}

In [ ]:
from keras.layers import BatchNormalization
def build_mlp2(input_shape, output_units=10, n_neurons=[512,256,128]):
    input_layer = keras.layers.Input(input_shape)
    x = BatchNormalization()(input_layer)
    for i, n_units in enumerate(n_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, activation='relu', name='hidden_layer'+str(i))(x)
        else:
            x = keras.layers.Dense(units=n_units, activation='relu', name='hidden_layer'+str(i))(x)
    
    out = keras.layers.Dense(units=output_units, activation='softmax', name='output')(x)
    model = keras.models.Model(inputs=input_layer, outputs=out)
    
    return model

In [ ]:
results2 = {}
for i, bat_size in enumerate(BATCH_SIZE):
    print('Experiment with batch size-%s' % str(bat_size))
    model = build_mlp1(input_shape=x_train.shape[1:])
    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.fit(x_train, y_train, batch_size=bat_size, epochs=EPOCHS, validation_data=(x_test, y_test), shuffle=True)

    train_loss = model.history.history['loss']
    valid_loss = model.history.history['val_loss']
    train_acc = model.history.history['accuracy']
    valid_acc = model.history.history['val_accuracy']

    exp_name_tag = 'batch size=-%s' % str(bat_size)
    results2[exp_name_tag] = {'train-loss': train_loss,
                             'val-loss': valid_loss,
                             'train-acc': train_acc,
                             'val-acc': valid_acc}